In [128]:
import tensorflow as tf
import numpy as np
from keras import layers, Model

In [129]:
inputs = np.arange(16).reshape(4,4)
# tf.constant(inputs) => crée un Tensor avec l'inputs
tensor_inputs = tf.constant(inputs)

In [132]:
def encode_haar(inputs):
    length = tf.shape(inputs)[0]
    # index = tf.cast(tf.divide(length, 2), tf.int64)
    
    indexes = tf.constant([[[0,0],[0,2],[2,0],[2,2]],
                           [[1,1],[1,3],[3,1],[3,3]],
                           [[0,1],[0,3],[2,1],[2,3]],
                           [[1,0],[1,2],[3,0],[3,2]]]) 
    
    slice_inputs = [tf.gather_nd(inputs, indexes[0]), tf.gather_nd(inputs, indexes[1]), tf.gather_nd(inputs, indexes[2]), tf.gather_nd(inputs, indexes[3])]
    slice_inputs = tf.reshape(slice_inputs, [length,2,2])
    
    tensor_average = tf.TensorArray(tf.int64, size=length, dynamic_size=False)
    for i in range(length):
        tensor_average = tensor_average.write(i, tf.reduce_mean(slice_inputs[i]))
    m_average = tf.reshape(tensor_average.stack(), [length,1])
    
    tensor_diff_rows = tf.TensorArray(tf.int64, size=length, dynamic_size=False)
    for i in range(length):
        tensor_diff_rows = tensor_diff_rows.write(i, tf.reduce_sum(slice_inputs[i][0,:]) - tf.reduce_sum(slice_inputs[i][1,:]))
    m_diff_rows = tf.reshape(tensor_diff_rows.stack(), [length,1])
    
    tensor_diff_cols = tf.TensorArray(tf.int64, size=length, dynamic_size=False)
    for i in range(length):
        tensor_diff_cols = tensor_diff_cols.write(i, tf.reduce_sum(slice_inputs[i][:,0]) - tf.reduce_sum(slice_inputs[i][:,1]))
    m_diff_cols = tf.reshape(tensor_diff_cols.stack(), [length,1])

    tensor_diff_diagonals = tf.TensorArray(tf.int64, size=length, dynamic_size=False)
    for i in range(length):
        tensor_diff_diagonals = tensor_diff_diagonals.write(i, slice_inputs[i][0,0] + slice_inputs[i][1,1] - slice_inputs[i][0,1] - slice_inputs[i][1,0])
    m_diff_diagonals = tf.reshape(tensor_diff_diagonals.stack(), [length,1]) 
        
    m_result = tf.TensorArray(tf.int64, size=0, dynamic_size=True)
    for i in range(0, length, 2):
        row_even = tf.concat([m_average[i], m_diff_rows[i], m_average[i+1], m_diff_rows[i+1]], 0)
        row_odd = tf.concat([m_diff_cols[i], m_diff_diagonals[i], m_diff_cols[i+1], m_diff_diagonals[i+1]], 0)
        m_result = m_result.write(i, row_even)
        m_result = m_result.write(i+1, row_odd)
    t = tf.reshape(m_result.stack(), [length, length])
    print(t)
    return t

In [133]:
encode_haar(tensor_inputs)

tf.Tensor(
[[  5 -16  10 -16]
 [ -4   0  -4   0]
 [  6 -16   9 -16]
 [ -4   0  -4   0]], shape=(4, 4), dtype=int64)


<tf.Tensor: shape=(4, 4), dtype=int64, numpy=
array([[  5, -16,  10, -16],
       [ -4,   0,  -4,   0],
       [  6, -16,   9, -16],
       [ -4,   0,  -4,   0]])>